In [1]:
import librosa as lr
import numpy as np
import scipy.signal as signal
import soundfile as sf
import matplotlib.pyplot as plt
from scipy.io import wavfile
import IPython.display as ipd
from AudioUtilsCustom import *




In [2]:
#from compression_custom import Audiopackage
#from compression_custom import AudioSegment
import time


In [3]:
filepath = 'C:\\Users\\chorj\\Downloads\\WavFiles'
input_file1 = filepath + '\\Hercules_dry_Drums_Kick.wav'
input_file2 = filepath + '\\Hercules_dry_Drums_Loop.wav'

# Read the wav file 
#samplerate1, data1 = wavfile.read(input_file1)
#samplerate2, data2 = wavfile.read(input_file2)
#print('Sampling rate = ', samplerate1)
#dt1, sr1 = lr.load(input_file1, sr = samplerate1)
#dt2, sr2 = lr.load(input_file2, sr = samplerate2)


In [22]:
from AudioUtilsCustom import *

class Audiopackage:
        
    def __init__(self, audiofile, parsetype = "spectrogram"):
        if parsetype != "exact" and parsetype != "similar" and parsetype != "spectrogram":
            print("Invalid parsetype")
            return
        self.name = extract_filename(audiofile)
        samplerate, audio = wavfile.read(audiofile)
        self.parsetype = parsetype
        #load audio as wavfile.
        self.samplerate = samplerate
        self.audio = audio
        #slice audio

        if self.parsetype == "spectrogram":
            self.lrfile, self.samplerate = lr.load(audiofile, sr = self.samplerate)
        
        self.audiosegmentlist = [] 
        self.audioIndex = find_blocks(audio) 
        self.length = len(self.audio)
        self.segmentSize = 0
        self.audioIndexSize = len(self.audioIndex)
        #print("AudioPackage Initialized")
        #print("Audio Length: ", self.length)
        #print("Audio Index length: ", self.audioIndex)
        self.parse()
    
    def lengthExists(self, i, duration):

        if duration == self.audiosegmentlist[i].length:
            #print("Length Exists")
            return True
        return False
    
    def lengthSimilar(self, i, duration):
        origin = self.audiosegmentlist[i].length
        if abs((duration - origin)/origin) < 0.1:
            #print("Length Similar")
            return True
        #print("Length Not Similar")
        return False
    
    def parse(self):
        #print(len(self.lrfile))
        #print("is the length of LR file")
        for i in range(self.audioIndexSize):
            if self.parsetype == "spectrogram":
                #print("Making it")
                target_lrfile = self.lrfile[self.audioIndex[i][0]:self.audioIndex[i][1]]
            #print(len(target_lrfile))
            #print("is the length of LR file")
            target = self.audio[self.audioIndex[i][0]:self.audioIndex[i][1]]
            start = self.audioIndex[i][0]
            end = self.audioIndex[i][1]
            duration = end - start
            if self.segmentSize == 0:
                #print("First Sample Added")
                if self.parsetype == "spectrogram":
                    self.audiosegmentlist.append(AudioSegment(target, duration, start, lrfile = self.lrfile[start:end], islibrosa=True))
                else:
                    self.audiosegmentlist.append(AudioSegment(target, duration, start))
                self.segmentSize += 1
            else:
                if self.parsetype == "exact":
                    self.apply_compare_exact(target, duration, start)
                elif self.parsetype == "similar":
                    self.apply_compare_similar(target, duration, start)
                elif self.parsetype == "spectrogram":
                    self.apply_compare_spectrogram(target, duration, start, target_lrfile)

    def apply_compare_exact(self, target, duration, start):

        for i in range(self.segmentSize):
            if self.lengthExists(i, duration) == True:
                if compare_exact(self.audiosegmentlist[i].audio, target) == True:
                    self.audiosegmentlist[i].append(start)
                    return
        #if no exact match, then append
        self.audiosegmentlist.append(AudioSegment(target, duration, start))
        self.segmentSize += 1

            
    def apply_compare_similar(self, target, duration, start):
        for i in range(self.segmentSize):
            if self.lengthSimilar(i, duration) == True:
                if compare_similar(self.audiosegmentlist[i].audio, target) > 0.99:
                    self.audiosegmentlist[i].append(start)
                    return
        self.audiosegmentlist.append(AudioSegment(target, duration, start))
        self.segmentSize += 1
    def apply_compare_spectrogram(self, target, duration, start, target_lrfile):
        for i in range(self.segmentSize):
            if self.lengthSimilar(i, duration) == True:
                if compare_spectrogram(self.audiosegmentlist[i].lrfile, target_lrfile) > 0.99:
                    self.audiosegmentlist[i].append(start)
                    return
        self.audiosegmentlist.append(AudioSegment(target, duration, start, lrfile = target_lrfile, islibrosa=True))
        self.segmentSize += 1
        # indexlist[9].append(index)
            #compare length first
            #if length is same, then compare each sample
        #else indexlist.append(AudioSegment(audio, length, index))



    def print(self):
        for i in range (len(self.audiosegmentlist)):
            print(self.audiosegmentlist[i])
    
    def sumSegments(self):
        ans = 0 
        for i in range(self.segmentSize):
            ans += self.audiosegmentlist[i].length
        return ans
    def compratio(self):
        return self.sumSegments()/self.length
    
    def printInfo(self):
        print("Audio Nane: ", self.name)
        print("Audio Length: ", self.length)
        #print("Audio Index length: ", self.audioIndex)
        print("Segment Size: ", self.segmentSize)
        #print("Audio Segment List: ", self.audiosegmentlist)
        print("Sum of Segments: ", self.sumSegments())
        print("Ratio of Compression: ", self.sumSegments()/self.length)


class AudioSegment:
    #처음 불릴때
    def __init__(self, audio, length, index, amplitude = 1, lrfile = None, islibrosa = False):
        self.indexlist = [index] 
        self.length = length
        self.audio = audio
        self.amplitude = amplitude

        self.lrfile = lrfile
    def __str__(self):
        return "SampleLength: " + str(self.length) + " , IndexList: " + str(self.indexlist)
    def append(self, index, amplitude = 1):
        #초기 샘플보다 후기 샘플이 더 adaptable 할 경우
        self.indexlist.append(index)

In [24]:
start_time = time.time()
example = Audiopackage(input_file1, "exact")
newtime = time.time() - start_time
print("Seconds taken: " + str(newtime))
example2 = Audiopackage(input_file2, "exact")
newtime2 = time.time() - newtime
print("Seconds taken: " + str(newtime2))
example3 = Audiopackage(input_file1, "similar")
newtime3 = time.time() - newtime2
print("Seconds taken: " + str(newtime3))
example4 = Audiopackage(input_file2, "similar")
newtime4 = time.time() - newtime3
print("Seconds taken: " + str(newtime4))
example5 = Audiopackage(input_file1, "spectrogram")
newtime5 = time.time() - newtime4
print("Seconds taken: " + str(newtime5))
example6 = Audiopackage(input_file2, "spectrogram")
newtime6 = time.time() - newtime5
print("Seconds taken: " + str(newtime6))

C:\Users\chorj\AppData\Local\Temp\ipykernel_12304\2617041215.py:10: WavFileWarning: Chunk (non-data) not understood, skipping it.
  samplerate, audio = wavfile.read(audiofile)


Seconds taken: 1.2450950145721436
Seconds taken: 1703711615.0153108
Seconds taken: 15.849816799163818


C:\Users\chorj\AppData\Local\Temp\ipykernel_12304\2617041215.py:10: WavFileWarning: Chunk (non-data) not understood, skipping it.
  samplerate, audio = wavfile.read(audiofile)


Seconds taken: 1703711625.7057002


C:\Users\chorj\AppData\Local\Temp\ipykernel_12304\2617041215.py:10: WavFileWarning: Chunk (non-data) not understood, skipping it.
  samplerate, audio = wavfile.read(audiofile)


Seconds taken: 16.813004732131958
Seconds taken: 1703711626.6264856


C:\Users\chorj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1115
  warnings.warn(
C:\Users\chorj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1103
  warnings.warn(
C:\Users\chorj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=845
  warnings.warn(
C:\Users\chorj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=865

In [25]:
example.printInfo()
example2.printInfo()
example3.printInfo()
example4.printInfo()
example5.printInfo()
example6.printInfo()

Audio Nane:  Hercules_dry_Drums_Kick.wav
Audio Length:  5534833
Segment Size:  33
Sum of Segments:  1403368
Ratio of Compression:  0.25355200418874424
Audio Nane:  Hercules_dry_Drums_Loop.wav
Audio Length:  5534833
Segment Size:  77
Sum of Segments:  3528482
Ratio of Compression:  0.6375046907467669
Audio Nane:  Hercules_dry_Drums_Kick.wav
Audio Length:  5534833
Segment Size:  15
Sum of Segments:  1077239
Ratio of Compression:  0.19462899783968188
Audio Nane:  Hercules_dry_Drums_Loop.wav
Audio Length:  5534833
Segment Size:  46
Sum of Segments:  3221101
Ratio of Compression:  0.5819689591357138
Audio Nane:  Hercules_dry_Drums_Kick.wav
Audio Length:  5534833
Segment Size:  15
Sum of Segments:  1077239
Ratio of Compression:  0.19462899783968188
Audio Nane:  Hercules_dry_Drums_Loop.wav
Audio Length:  5534833
Segment Size:  32
Sum of Segments:  3113021
Ratio of Compression:  0.5624417213672029


In [30]:
def find_non_zero_ratio(sample):
    count = 0
    for i in range(len(sample)):
        if sample[i][0] == 0 and sample[i][1] == 0:
            count += 1
    return round(1 - count/len(sample),4)


In [27]:
import soundfile as sf
import zipfile
import os
import time

def compare_compression_ratio(input_file):
    # Specify the output file paths
    flac_output_file = 'compressed.flac'
    zip_output_file = 'compressed.zip'

    # Read the WAV file
    data, samplerate = sf.read(input_file)

    start_time = time.time()
    # Write the FLAC file
    sf.write(flac_output_file, data, samplerate, format='flac')
    # time
    flac_time = time.time() - start_time

    start_time = time.time()
    # Create a new ZIP file
    with zipfile.ZipFile(zip_output_file, 'w', compression=zipfile.ZIP_DEFLATED) as zipf:
        # Add the WAV file to the ZIP file
        zipf.write(input_file, arcname='audio.wav')
    # time
    zip_time = time.time() - start_time

    # Get the file sizes
    input_size = os.path.getsize(input_file)
    flac_size = os.path.getsize(flac_output_file)
    zip_size = os.path.getsize(zip_output_file)

    start_time = time.time()
    tryExact = Audiopackage(input_file, "exact")
    exact_time = time.time() - start_time

    start_time = time.time()
    trySimilar = Audiopackage(input_file, "similar")
    similar_time = time.time() - start_time

    start_time = time.time()
    trySpectrogram = Audiopackage(input_file, "spectrogram")
    spectrogram_time = time.time() - start_time
    
    spectrogram_ratio = trySpectrogram.compratio()
    similar_ratio = trySimilar.compratio()
    exact_ratio = tryExact.compratio()
    flac_ratio = flac_size / input_size
    zip_ratio = zip_size / input_size
    ratiofamily = [zip_ratio, flac_ratio, exact_ratio, similar_ratio, spectrogram_ratio]
    timefamily = [zip_time, flac_time, exact_time, similar_time, spectrogram_time]
    non_zero_ratio = find_non_zero_ratio(data)
    baseline = non_zero_ratio
    
    # Calculate the compression ratios


    #print("Ratio of non-zero: " + str(non_zero_ratio))
    # Print the results
    #print(f"FLAC Compression Ratio: {flac_ratio:.4f}")
    #print(f"ZIP Compression Ratio: {zip_ratio:.4f}")
    
    #TODO: print time

    # Delete the temporary files
    os.remove(flac_output_file)
    os.remove(zip_output_file)
    return [ratiofamily, timefamily, baseline]


In [31]:
def resultanalysis(ratiofamily, timefamily, baseline):
    print("Baseline: " + str(baseline))
    print("ZIP Compression Ratio: " + str(ratiofamily[0]))
    print("FLAC Compression Ratio: " + str(ratiofamily[1]))
    print("Exact Compression Ratio: " + str(ratiofamily[2]))
    print("Similar Compression Ratio: " + str(ratiofamily[3]))
    print("Spectrogram Compression Ratio: " + str(ratiofamily[4]))
    print("ZIP Compression Time: " + str(timefamily[0]))
    print("FLAC Compression Time: " + str(timefamily[1]))
    print("Exact Compression Time: " + str(timefamily[2]))
    print("Similar Compression Time: " + str(timefamily[3]))
    print("Spectrogram Compression Time: " + str(timefamily[4]))

a = compare_compression_ratio(input_file1)
resultanalysis(a[0], a[1], a[2])

C:\Users\chorj\AppData\Local\Temp\ipykernel_12304\2617041215.py:10: WavFileWarning: Chunk (non-data) not understood, skipping it.
  samplerate, audio = wavfile.read(audiofile)
C:\Users\chorj\AppData\Local\Temp\ipykernel_12304\2617041215.py:10: WavFileWarning: Chunk (non-data) not understood, skipping it.
  samplerate, audio = wavfile.read(audiofile)


Baseline: 0.4512
ZIP Compression Ratio: 0.36612599549920144
FLAC Compression Ratio: 0.13914049406402765
Exact Compression Ratio: 0.25355200418874424
Similar Compression Ratio: 0.19462899783968188
Spectrogram Compression Ratio: 0.19462899783968188
ZIP Compression Time: 0.4221458435058594
FLAC Compression Time: 0.18966054916381836
Exact Compression Time: 1.2718048095703125
Similar Compression Time: 13.094358682632446
Spectrogram Compression Time: 0.9500701427459717


In [32]:
a = compare_compression_ratio(input_file2)
resultanalysis(a[0], a[1], a[2])

C:\Users\chorj\AppData\Local\Temp\ipykernel_12304\2617041215.py:10: WavFileWarning: Chunk (non-data) not understood, skipping it.
  samplerate, audio = wavfile.read(audiofile)
C:\Users\chorj\AppData\Local\Temp\ipykernel_12304\2617041215.py:10: WavFileWarning: Chunk (non-data) not understood, skipping it.
  samplerate, audio = wavfile.read(audiofile)
C:\Users\chorj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1115
  warnings.warn(
C:\Users\chorj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1103
  warnings.warn(
C:\Users\chorj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages

Baseline: 0.6603
ZIP Compression Ratio: 0.5855697053751763
FLAC Compression Ratio: 0.24142269775819472
Exact Compression Ratio: 0.6375046907467669
Similar Compression Ratio: 0.5819689591357138
Spectrogram Compression Ratio: 0.5624417213672029
ZIP Compression Time: 0.7810037136077881
FLAC Compression Time: 0.1998438835144043
Exact Compression Time: 0.7623767852783203
Similar Compression Time: 10.041547536849976
Spectrogram Compression Time: 1.02325439453125


In [ ]:
import glob

# Specify the folder path
folder_path = 'C:\\Users\\chorj\\Downloads\\WavFiles\\Hercules_Dry(AbelEdit)'

# Find all WAV files in the folder
wav_files = glob.glob(folder_path + '/*.wav')

# Print the list of WAV files
for file in wav_files:
    print(file)


In [ ]:
def analyze_folder(folder_path):
    # Find all WAV files in the folder
    wav_files = glob.glob(folder_path + '/*.wav')
    resultList = []
    # Print the list of WAV files
    for file in wav_files:
        print(file)
        resultList.append(compare_compression_ratio(file))
    return [resultList]


In [ ]:
resultList = analyze_folder(folder_path)    

C:\Users\chorj\Downloads\WavFiles\Hercules_Dry(AbelEdit)\Hercules_dry_Bass_BassTop.wav
Ratio of non-zero: 0.73
FLAC Compression Ratio: 0.0889
ZIP Compression Ratio: 0.5792
C:\Users\chorj\Downloads\WavFiles\Hercules_Dry(AbelEdit)\Hercules_dry_Bass_Sub.wav
Ratio of non-zero: 0.7873
FLAC Compression Ratio: 0.0577
ZIP Compression Ratio: 0.7135
C:\Users\chorj\Downloads\WavFiles\Hercules_Dry(AbelEdit)\Hercules_dry_Brass_Horns.wav
Ratio of non-zero: 0.8507
FLAC Compression Ratio: 0.1985
ZIP Compression Ratio: 0.7520
C:\Users\chorj\Downloads\WavFiles\Hercules_Dry(AbelEdit)\Hercules_dry_Brass_Low Brs.wav
Ratio of non-zero: 0.8055
FLAC Compression Ratio: 0.1624
ZIP Compression Ratio: 0.7061
C:\Users\chorj\Downloads\WavFiles\Hercules_Dry(AbelEdit)\Hercules_dry_Brass_Trombone.wav
Ratio of non-zero: 0.1763
FLAC Compression Ratio: 0.0406
ZIP Compression Ratio: 0.1430
C:\Users\chorj\Downloads\WavFiles\Hercules_Dry(AbelEdit)\Hercules_dry_Brass_Trumpet.wav
Ratio of non-zero: 0.7558
FLAC Compression Rat

In [ ]:
non_zero_ratio = 0
flac_ratio = 0
zip_ratio = 0
for i in range(len(resultList)):
    non_zero_ratio += float(resultList[i][0])
    flac_ratio += float(resultList[i][1])
    zip_ratio += float(resultList[i][2])

print("Average non-zero ratio: " + str(non_zero_ratio/len(resultList)))
print("Average flac ratio: " + str(flac_ratio/len(resultList)))
print("Average zip ratio: " + str(zip_ratio/len(resultList)))

print("Zip to non-zero Compression ratio: " + str(zip_ratio/non_zero_ratio))
print("FLAC to non-zero Compression ratio: " + str(flac_ratio/non_zero_ratio))
print("Zip to FLAC Compression ratio: " + str(flac_ratio/zip_ratio))

Average non-zero ratio: 0.75865
Average flac ratio: 0.07329174293944803
Average zip ratio: 0.6463672717099354
Zip to non-zero Compressoin ratio: 0.8519966673827658
FLAC to non-zero Compressoin ratio: 0.09660811037955318
Zip to FLAC Compressoin ratio: 0.11339024444346948
